In [1]:
with open('keys.txt', 'r') as file:
    # Read the entire contents of the file into a variable
    file_contents = file.read()
    keys = file_contents.split('\n')

import praw

reddit = praw.Reddit(
    client_id=keys[0],
    client_secret=keys[1],
    user_agent=keys[2],
    username=keys[3],
    password=keys[4]
)

In [2]:
subreddit = reddit.subreddit('wallstreetbets')
for post in subreddit.new(limit=10):
    print(post.title)

Wash sale - advice needed
Volume tickers in play
Every regional bank Chief Risk Officer in 2023 :
Daily Discussion Thread for May 19, 2023
US Treasury's cash balance falls to $68.3bn, lowest since 2021.
Corporate wants you to find the difference between these two pictures
Minor win before my soon-to-be McDonalds job orientation
Rate hike expectations just rose, JPow to come out today to shock the markets?
FED rates are going down this fall
The end of the world is bullish.


In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [4]:
for post in subreddit.new(limit=10):
    doc = nlp(post.title)
    for token in doc:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_)

Wash wash VERB VB compound
sale sale NOUN NN compound
- - PUNCT HYPH punct
advice advice NOUN NN nsubj
needed need VERB VBD ROOT
Volume volume NOUN NN nsubj
tickers ticker VERB VBZ ROOT
in in ADP IN prep
play play NOUN NN pobj
Every every DET DT det
regional regional ADJ JJ amod
bank bank NOUN NN compound
Chief Chief PROPN NNP compound
Risk Risk PROPN NNP compound
Officer Officer PROPN NNP ROOT
in in ADP IN prep
2023 2023 NUM CD pobj
: : PUNCT : punct
Daily Daily PROPN NNP compound
Discussion Discussion PROPN NNP compound
Thread Thread PROPN NNP ROOT
for for ADP IN prep
May May PROPN NNP pobj
19 19 NUM CD nummod
, , PUNCT , punct
2023 2023 NUM CD nummod
US US PROPN NNP compound
Treasury Treasury PROPN NNP poss
's 's PART POS case
cash cash NOUN NN compound
balance balance NOUN NN nsubj
falls fall VERB VBZ ROOT
to to ADP IN prep
$ $ SYM $ nmod
68.3bn 68.3bn NUM CD pobj
, , PUNCT , punct
lowest low ADJ JJS conj
since since SCONJ IN prep
2021 2021 NUM CD pobj
. . PUNCT . punct
Corporate c

In [5]:
def extract_stock_information(text):
    doc = nlp(text)
    entities = []
    sentiments = []

    for entity in doc.ents:
        if entity.label_ == "ORG" and "tesla" in entity.text.lower():
            entities.append(entity.text)

    for sentence in doc.sents:
        sentiment = sentence.sentiment
        sentiments.append(sentiment)

    return entities, sentiments

In [6]:
for post in subreddit.new(limit=10):
    entities, sentiments = extract_stock_information(post.title)
    if entities:
        print("Entities: ", entities)
        print("Sentiments: ", sentiments)


In [7]:
import yfinance as yf

def retrieve_historical_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    return stock_data

In [8]:
import datetime as datetime

symbol = "TSLA"
start_date = datetime.datetime(2023, 4, 1)
end_date = datetime.datetime(2023, 5, 1)

stock_data = retrieve_historical_stock_data(symbol, start_date, end_date)
stock_data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-04-03,199.910004,202.690002,192.199997,194.770004,194.770004,169545900
2023-04-04,197.320007,198.740005,190.320007,192.580002,192.580002,126463800
2023-04-05,190.520004,190.679993,183.759995,185.520004,185.520004,133882500
2023-04-06,183.080002,186.389999,179.740005,185.059998,185.059998,123857900
2023-04-10,179.940002,185.100006,176.110001,184.509995,184.509995,142154600


In [31]:
posts = []

for post in subreddit.new(limit = 100):
    # if start_date.timestamp() <= post.created_utc <= end_date.timestamp():
    posts.append({
        'title': post.title,
        'body': post.selftext,
        'date': post.created_utc,
        'score': post.score
    })
    # elif post.created_utc < start_date.timestamp():
    #     break

import pandas as pd
from datetime import datetime

posts_df = pd.DataFrame(posts)
posts_df = posts_df.assign(Date = lambda x: datetime.fromtimestamp(x['date'][0]))
posts_df = posts_df.assign(Date = lambda x: x['Date'][0].strftime("%Y-%m-%d"))
posts_df.drop()
posts_df.head()

,title,body,date,score,Date
0,The state comptroller recommends that the MTA ...,,1.684500e+09,1,2023-05-19
1,Do opex days tend to be bullish or bearish?,I'm trying to understand what opex means for m...,1.684500e+09,1,2023-05-19
2,Volume tickers in play,Trading Thesis: Tracking the float along with ...,1.684497e+09,2,2023-05-19
3,Every regional bank Chief Risk Officer in 2023 :,,1.684495e+09,8,2023-05-19
4,"Daily Discussion Thread for May 19, 2023",**Come hang out with us on the NEW WallStreetB...,1.684490e+09,29,2023-05-19


In [32]:
def merge_data(posts, stock_data):
    merged_data = pd.merge(posts, stock_data, how='inner', left_on='Date', right_index=True)
    return merged_data

In [33]:
merged_data = merge_data(posts_df, stock_data)
print(merged_data)

ValueError: You are trying to merge on object and datetime64[ns] columns. If you wish to proceed you should use pd.concat

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Assume merged_data contains the merged DataFrame with Reddit posts and stock price data
# Ensure the DataFrame has columns for 'date', 'title', 'body', 'score', 'Open', 'Close', etc.

# Prepare the features and target variables
features = merged_data[['score', 'Open', 'Volume', 'Date']]  # Adjust the features based on your requirements
target = merged_data['Close']  # Adjust the target variable based on your requirements

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

# Print the model performance
print("Train Score:", train_score)
print("Test Score:", test_score)